In [1]:
## Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os
import ast
import alive_progress
from alive_progress import alive_bar

# # set inline print
# %matplotlib inline

rootdir = os.path.join(".")
path_logs = os.path.join(rootdir,"learning_logs.csv")

def apfd(nqueries: list, hypsize: list, max_nqueries=None):
    hs = hypsize
    nq = nqueries
    assert hs[0] == 1
    assert len(nq) == len(hs)
    
    extra_nqueries, extra_hs = [],[]
    if(not max_nqueries is None): 
        extra_nqueries = [max_nqueries]
        extra_hs = [hs[-1]]

    return 1 - (np.sum(np.multiply([*nq,*extra_nqueries], np.diff([0, *hs,*extra_hs]))) / (np.max([*nq,*extra_nqueries]) * np.max(hs))) + (1.0 / (2 * np.max([*nq,*extra_nqueries])))

def auc_learning(nqueries: list, hypsize: list, max_nqueries=None):
    hs = hypsize
    nq = nqueries
    assert hs[0] == 1
    assert len(nq) == len(hs)
    
    extra_nqueries, extra_hs = [],[]
    if(not max_nqueries is None): 
        extra_nqueries = [max_nqueries]
        extra_hs = [hs[-1]]
    
    return np.trapz([*hs,*extra_hs],[*nq,*extra_nqueries])

def derive_data(data_frame: pd.DataFrame.dtypes):
    # first, copy dataframe
    df = data_frame.copy()
    
    # ...start by concatenating CTT name with number of extra states
    df['CTT_ES']  = df['CTT']+"("+df['Extra States'].astype(str)+")"
    
    # ... and then split queries/symbols into different columns
    for qtype in ["Learning", "Testing"]:
        _lst= df[f"{qtype} queries/symbols"].apply(lambda x: [i.split('/') for i in ast.literal_eval(x)])
        df[f"{qtype} queries"]            = _lst.apply(lambda x : np.cumsum([int(i[0]) for i in x])) # resets
        df[f"{qtype} symbols"]            = _lst.apply(lambda x : np.cumsum([int(i[1]) for i in x])) # symbols w/o resets

    # ... and then parsing string with hypotheses sizes as array of integers
    df["HypSize"] = df["HypSize"].apply(lambda x: ast.literal_eval(x)) 
    df["TQ [Symbols]"] = df["EQ [Symbols]"]+df["MQ [Symbols]"]
    df["TQ [Resets]"] = df["EQ [Resets]"]+df["MQ [Resets]"]

    # ... and then append qSize to HypSize, if the run is successfull 
    df["HypSize"] = df.apply(lambda x: x.HypSize + [x.Qsize] if x.Equivalent=='OK' and len(x.HypSize) < x.Rounds else x.HypSize, axis=1)
    
    # ... and then include #EQs from the single-state model
    df["Testing queries"] = df["Testing queries"].apply(lambda x: [0,*x])
    df["Testing symbols"] = df["Testing symbols"].apply(lambda x: [0,*x])
    
    # ... and then calculate the total number of queries
    df["Total queries"] = df.apply(lambda x: np.add(x["Testing queries"],x["Learning queries"]) if x.Equivalent=='OK' else [], axis=1)
    df["Total symbols"] = df.apply(lambda x: np.add(x["Testing symbols"],x["Learning symbols"]) if x.Equivalent=='OK' else [], axis=1)
    
    # ... and then (FINALLY!) calculate the APFD and AUC for EQs, and TQs
    df_eq = df.query('`Equivalent`=="OK"')
    
    the_cols = ["SUL name","TQ [Symbols]","EQ [Symbols]"]
    max_eqs = df[the_cols].groupby(["SUL name"]).max().to_dict()

    df["APFD_testing"] = df.apply(lambda x: apfd(x['Testing symbols'],x['HypSize']) if x.Equivalent=='OK' else -1, axis=1)
    df["APFD_total"] = df.apply(lambda x: apfd(x['Total symbols'],x['HypSize']) if x.Equivalent=='OK' else -1, axis=1)
    
    df["APFDx_testing"] = df.apply(lambda x: apfd(x['Testing symbols'],x['HypSize'],max_nqueries=max_eqs['EQ [Symbols]'][x['SUL name']]) if x.Equivalent=='OK' else -1, axis=1)
    df["APFDx_total"] = df.apply(lambda x: apfd(x['Total symbols'],x['HypSize'],max_nqueries=max_eqs['TQ [Symbols]'][x['SUL name']]) if x.Equivalent=='OK' else -1, axis=1)
    
    df["AUC_testing"] = df.apply(lambda x: auc_learning(x['Testing symbols'],x['HypSize'],max_nqueries=max_eqs['EQ [Symbols]'][x['SUL name']]) if x.Equivalent=='OK' else -1, axis=1)
    df["AUC_total"] = df.apply(lambda x: auc_learning(x['Total symbols'],x['HypSize'],max_nqueries=max_eqs['TQ [Symbols]'][x['SUL name']]) if x.Equivalent=='OK' else -1, axis=1)
    
    # to close, return the new dataframe with derived columns
    return df

def _interp_addsorted(alist, datapoints=[]):
    cc_dp = alist.copy()
    for newdp in datapoints:
        if(newdp in cc_dp): continue
        cc_dp = np.insert(cc_dp,np.searchsorted(cc_dp,newdp),newdp)
    return cc_dp

def interp(data: pd.DataFrame.dtypes, col_costs: str, col_hypsizes: str, datapoints=[]):
    df_subset = data.copy()
    df_subset[col_hypsizes+'_withdatapoints']=df_subset[col_hypsizes].apply(lambda x: _interp_addsorted(x,datapoints))
    df_subset[col_costs]=df_subset.apply(lambda x: np.interp(x[col_hypsizes+'_withdatapoints'], x[col_hypsizes], x[col_costs]),axis=1)
    df_subset[col_hypsizes]=df_subset[col_hypsizes+'_withdatapoints']
    df_subset.drop(col_hypsizes+'_withdatapoints',inplace=True,axis=1)
    return df_subset

# Load the CSV file and derive new columns for APFD, APFDx, and AUC

In [2]:
df = pd.read_csv(path_logs)
df = derive_data(df)
# df.columns

# Setup the list of SULs to be analyzed

In [3]:
df_equiv=df.query(f'`Equivalent`=="OK"').sort_values(by=['APFDx_testing'],ascending=False)

all_qtype = ['Testing symbols', 'Total symbols', 'Testing queries', 'Total queries'] 
# all_qtype = ['Testing symbols'] # alternative
all_sulname = df['SUL name'].drop_duplicates().sort_values()
total = len(all_qtype)*len(all_sulname)

# Plot %states detected per test case (for all methods)

In [4]:
# define figure size
sns.set(rc={'figure.figsize':(10,5),'figure.dpi':200})

with alive_bar(total, force_tty=True, title=f'Plotting APFD') as bar:
    for qtype in all_qtype:
        for sulname in all_sulname:
            # get one entry
            subj=df_equiv.query(f'`SUL name`=="{sulname}"').copy() 

            # add percent columns
            subj['HypSizePercent'] = subj['HypSize'].apply(lambda x: x/np.max(x)*100)

            # explode column with % of symbols and hypothesis sizes in the learning process
            subj=subj.explode(['HypSizePercent',f'{qtype}'])

            #create line chart
            apfd_plot = sns.lineplot(subj, x=f'{qtype}', y='HypSizePercent',
                                     markers=True, 
                                     #style='Extra States', hue='CTT',
                                     style='CTT_ES', hue='CTT_ES',
                                     palette='rocket'
                                    )
            apfd_plot.set(xscale='log')
            locator = ticker.LogLocator()
            locator.MAXTICKS = np.max(subj[f'{qtype}'])
            apfd_plot.xaxis.set_major_locator(locator)

            apfd_plot.yaxis.set_major_locator(ticker.MultipleLocator(10))
            apfd_plot.set_ylim(0,100)

            #add plot labels, titles and legends
            plt.xlabel(f'Number of {qtype}')
            plt.ylabel('Fraction of the SUL learned')
            plt.legend(title='Testing Technique', loc='lower right')
            plt.title(f'Subject: {sulname}')

            # save line chart
            fig = apfd_plot.get_figure()
            os.makedirs(f'img/cumulative/{qtype}/', exist_ok=True)
            fig.savefig(f'img/cumulative/{qtype}/'+sulname.replace('.dot',f'_cumulative_{qtype}.jpg'))
            fig.clf()
            bar()

Plotting APFD |████████████████████████████████████████| 184/184 [100%] in 1:21.5 (2.26/s)                              


<Figure size 2000x1000 with 0 Axes>

# Plot heatmap of #test required to detect %states (for all methods)

In [5]:
# define figure size
sns.set(rc={'figure.figsize':(20,5)})

# define datapoints to interpolate 
new_dps = list(range(10,101,5))
new_dps.reverse()
with alive_bar(total, force_tty=True, title=f'Plotting heatmaps') as bar:
    for qtype in all_qtype:
        #print(qtype)
        for sulname in all_sulname:
            #print('\t',sulname)
            # get one entry
            subj=df_equiv.query(f'`SUL name`=="{sulname}"').copy() 

            # add percent columns
            subj['HypSizePercent'] = subj['HypSize'].apply(lambda x: x/np.max(x)*100)

            subj_interp=interp(data=subj, col_costs=f'{qtype}',col_hypsizes='HypSizePercent', datapoints=new_dps)
            subj_interp=subj_interp.explode([f'{qtype}','HypSizePercent'])
            subj_interp=subj_interp.query('HypSizePercent in @new_dps').sort_values(by=[f'{qtype}'],ascending=False)

            cols_order = subj_interp['CTT_ES'].drop_duplicates().to_list()
            cols_order.reverse()

            the_cols = ['SUL name','HypSizePercent','CTT_ES']
            subj_gb=subj_interp[the_cols+[f'{qtype}',]].groupby(the_cols).first()

            ## The scope of these changes made to
            ## pandas settings are local to with statement.
            #with pd.option_context('display.max_rows', None,
            #                       'display.max_columns', None,
            #                       'display.precision', 3,
            #                       'display.float_format', '{:,.1f}'.format,
            #                       ):
            #    next
            #    display(subj_gb.pivot_table(f'{qtype}','HypSizePercent',['SUL name','CTT_ES'])[[['BitVise.dot',it] for it in cols_order]])
            #    display(subj_gb)
            
            subj_pvt = subj_gb.pivot_table(f'{qtype}',['CTT_ES'],'HypSizePercent').sort_values(by=list(new_dps))
            subj_pvt = subj_pvt/subj_pvt.max().max()
            heatmap_cost=sns.heatmap(subj_pvt,
                                     #annot=True, fmt=",.7f", 
                                     cmap= sns.cm.rocket_r, linewidth=.5, linecolor='black'
                                    )
            
            #add plot labels, titles and legends
            plt.xlabel('Fraction of the SUL learned')
            plt.ylabel('')
            #plt.legend(title='Testing Technique', loc='lower right')
            plt.title(f'Fraction of {qtype} ({sulname})')

            
            # save line chart
            fig = heatmap_cost.get_figure()
            os.makedirs(f'img/heatmap/{qtype}/', exist_ok=True)
            fig.savefig(f'img/heatmap/{qtype}/'+sulname.replace('.dot',f'_heatmap_{qtype}.jpg'))
            fig.clf()
            bar()

Plotting heatmaps |████████████████████████████████████████| 184/184 [100%] in 1:13.8 (2.49/s)                          


<Figure size 4000x1000 with 0 Axes>